<a href="https://colab.research.google.com/github/EC-520-Deep-Learning-Project/ArtStyleDetector/blob/main/Image_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# For Transformations of Images (enter y in prompt)
!pip uninstall imgaug
!pip install imgaug==0.4.0


In [ ]:
%%capture
!pip install imagecorruptions

In [ ]:
# Imports 

import numpy as np
import os
import glob
import random
import imgaug as ia
import imgaug.augmenters as iaa
from PIL import Image
import cv2
import re
import imagecorruptions
from tqdm import tqdm 
import shutil
import math

In [ ]:
# Other Directory Info
colab = True
if colab:
  #Mounting Drive
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount = True)
  ART_DIR = '/content/gdrive/Shareddrives/520 Project/Data/wikipaintings_full/wikipaintings_train'
  CSV_PATH = '/content/gdrive/Shareddrives/520 Project/Data/efficientnetv2_csv'
  AUG_PATH = '/content/gdrive/Shareddrives/520 Project/Data/wikipaintings_full_aug/train'
else: #scc paths
  ART_DIR = '/projectnb/dl523/students/colejh/520/wikipaintings_full/wikipaintings_val'
  CSV_PATH = '/content/gdrive/Shareddrives/520 Project/Data/efficientnetv2_csv'
  AUG_PATH = '/projectnb/dl523/students/project/Sarcasm/content/gdrive/Shareddrives/520 Project/Data/wikipaintings_full_aug/train'

# Checking whether the augmenting path already exists
if not os.path.exists(AUG_PATH):
  os.makedirs(AUG_PATH)
  print("Directory at ",AUG_PATH,"has been created")


Mounted at /content/gdrive


In [ ]:
# Working Directory
small = False
# %%capture
%cd '.'
%pwd
if small:
  %cd 'gdrive/Shareddrives/520 Project/Data/wikipaintings_small/wikipaintings_train'
else:
  %cd 'gdrive/Shareddrives/520 Project/Data/wikipaintings_full/wikipaintings_train'
# %ls 


/content
/content/gdrive/Shareddrives/520 Project/Data/wikipaintings_full/wikipaintings_train


In [ ]:
# Getting the number of images present for each style in the current image folder
styles = [ style for style in os.listdir(ART_DIR) if os.path.isdir(os.path.join(ART_DIR, style)) ]
styles = sorted(styles)
# Cleaning up Label Names for cleaner diagrams later on
styles_for_labels = [re.sub(r'[^A-Za-z0-9 -]+', ' ', i) for i in styles]

# Number of images needed for balanced classes
file_count = []
for style in styles:
  number = sum(len(files) for _, _, files in os.walk(ART_DIR+'/'+style))
  file_count.append(number)
max_style_count = max(file_count);
num_for_parity = max_style_count-np.array(file_count)
for style,parity in zip(styles_for_labels,num_for_parity):
  print(style,' needs ',parity,' more examples')

Abstract Art  needs  9038  more examples
Abstract Expressionism  needs  8160  more examples
Art Informel  needs  9068  more examples
Art Nouveau Modern   needs  6481  more examples
Baroque  needs  6620  more examples
Color Field Painting  needs  8834  more examples
Cubism  needs  8487  more examples
Early Renaissance  needs  8776  more examples
Expressionism  needs  5120  more examples
High Renaissance  needs  8820  more examples
Impressionism  needs  0  more examples
Magic Realism  needs  9035  more examples
Mannerism Late Renaissance   needs  8888  more examples
Minimalism  needs  8827  more examples
Naive Art Primitivism   needs  8212  more examples
Neoclassicism  needs  7667  more examples
Northern Renaissance  needs  7905  more examples
Pop Art  needs  8938  more examples
Post-Impressionism  needs  4994  more examples
Realism  needs  1681  more examples
Rococo  needs  8291  more examples
Romanticism  needs  4532  more examples
Surrealism  needs  5966  more examples
Symbolism  need

In [ ]:
# Augmentations

seq = iaa.Sequential([
    iaa.Resize(256), # Smaller file size
    iaa.Fliplr(0.5), # Horizontal flips
    iaa.Flipud(0.2), # Vertical Flips
    iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.5)), iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-30, 30),
        shear=(-8, 8),
        cval=(0, 255)
  )
    ),
    # Common distortions one might see (pixelation, compression, dirty lens, out of focus)
    iaa.Sometimes(.8,iaa.OneOf([
                        iaa.imgcorruptlike.Pixelate(severity=1),
                        iaa.imgcorruptlike.JpegCompression(severity=1),
                        iaa.imgcorruptlike.Spatter(severity=1),
                        iaa.imgcorruptlike.ZoomBlur(severity=1)
                    ]),),
    iaa.SomeOf((0, 2),
            [
                # Blur the image 
                iaa.OneOf([
                    iaa.GaussianBlur((0, 4.0)),
                    iaa.AverageBlur(k=(2, 5)),
                    iaa.MedianBlur(k=(3, 11)),
                ]),

                # Sharpen
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),

                # Add noise
                iaa.AdditiveGaussianNoise(
                    loc=0, scale=(0.0, 0.15*255), per_channel=0.5
                ),
                # Randomly drop pixels
                iaa.Dropout((0.01, 0.1), per_channel=0.5),

                # Different type of noise 
                iaa.SaltAndPepper(0.15),
             
                # Invert color channels
                iaa.Invert(0.05, per_channel=True), # invert color channels

                # Add fixed value to pixels
                iaa.Add((-15, 15), per_channel=0.5),

                # Change brightness of images
                iaa.Multiply((0.5, 2.5), per_channel=0.5),

                # Adjust Contrast
                iaa.LinearContrast((0.5, 2.0), per_channel=0.5),

            ],
            # do the above augmentations in random order
            random_order=True
        )
    ], random_order=True) # apply all augmenters in random order


In [ ]:
def augment_and_save(ART_DIR,AUG_PATH,style,parity,max_style_count,batch =50,img_batch = 200,save_percent = .7):
  new_path = os.path.join(AUG_PATH, style)
  style_path = os.path.join(ART_DIR, style)
  # Create new directory in augmented folder
  if not os.path.exists(new_path):
    os.makedirs(new_path)

  # Change into current art style directory
  os.chdir(style_path)

  # List of all images in given style directory
  images = glob.glob("*.jpg") 
    
  if parity == 0: #Augmentations for the max class
    count = 1
    num_to_save = math.ceil(max_style_count*save_percent)
    print(num_to_save)
    for img in images:
      if count <= num_to_save:
        shutil.copy2(os.path.join(style_path,img),new_path)
      else:
        break
      count+=1
    
    # Subset of images to modify
    images = images[num_to_save-1:]

    # Number of images to augment
    remaining_images = max_style_count - count+1

    saved = 0
    while saved != remaining_images: 
      img_list = []
      img_names = []
      # Dont run past the right amount of augmented images
      if remaining_images-saved<img_batch:
        img_batch = remaining_images-saved
      for j in range(img_batch): 
        # Random list of images to be augmented 
        random_image = random.choice(images)
        im = cv2.imread(random_image)
        img_list.append(im)
        img_names.append(os.path.splitext(random_image)[0])

      if img_list:

        #Split into batches
        batch_list = [img_list[i:i + batch] for i in range(0, len(img_list), batch)]
        batch_names = [img_names[i:i + batch] for i in range(0, len(img_names), batch)]

        # Image augmentation
        for k in range(len(batch_list)):
          altered_images = seq(images = batch_list[k])

          # Store image in new aug folder
          for i in range(len(batch_list[k])):
            # filename = batch_names[k][i]

            # while os.path.exists(os.path.join(new_path,filename+'.jpg')):
            #   filename = filename +'aug'
            filename = str(saved)
            cv2.imwrite(os.path.join(new_path , filename+'.jpg'),altered_images[i])
            saved+=1

  else: # Save all copies of smaller classes, augment images until classes are balanced
    for fname in images:
      # copying the files to the destination directory
      shutil.copy2(os.path.join(style_path,fname),new_path)

    print('\nAugmenting ',style,' images...')
    saved = 0
    while saved != parity: 
      img_list = []
      img_names = []
      # Dont run past the right amount of augmented images
      if parity-saved<img_batch:
        img_batch = parity-saved
      for j in range(img_batch): 
        # Random list of images to be augmented 
        random_image = random.choice(images)
        im = cv2.imread(random_image)
        img_list.append(im)
        img_names.append(os.path.splitext(random_image)[0])

      if img_list:

        #Split into batches
        batch_list = [img_list[i:i + batch] for i in range(0, len(img_list), batch)]
        batch_names = [img_names[i:i + batch] for i in range(0, len(img_names), batch)]

        # Image augmentation
        for k in range(len(batch_list)):
          altered_images = seq(images = batch_list[k])

          # Store image in new aug folder
          for i in range(len(batch_list[k])):
            # filename = batch_names[k][i]

            # while os.path.exists(os.path.join(new_path,filename+'.jpg')):
            #   filename = filename +'aug'
            filename = str(saved)
            cv2.imwrite(os.path.join(new_path , filename+'.jpg'),altered_images[i])
            saved+=1



In [ ]:
# Augmenting and saving for each art style
for style, parity in zip(styles,num_for_parity):
  augment_and_save(ART_DIR,AUG_PATH,style,parity,max_style_count)


Augmenting  Naive_Art_(Primitivism)  images...


In [ ]:
#@title
styles = [ style for style in os.listdir(AUG_PATH) if os.path.isdir(os.path.join(AUG_PATH, style)) ]
styles = sorted(styles)
# Cleaning up Label Names for cleaner diagrams later on
styles_for_labels = [re.sub(r'[^A-Za-z0-9 -]+', ' ', i) for i in styles]

# Number of images needed for balanced classes
file_count = []
for style in styles:
  number = sum(len(files) for _, _, files in os.walk(AUG_PATH+'/'+style))
  file_count.append(number)
max_style_count = max(file_count);
num_for_parity = max_style_count-np.array(file_count)
for style,parity in zip(styles_for_labels,num_for_parity):
  print(style,' needs ',parity,' more examples')

Abstract Art  needs  0  more examples
Abstract Expressionism  needs  0  more examples
Art Informel  needs  0  more examples
Art Nouveau Modern   needs  0  more examples
Baroque  needs  0  more examples
Color Field Painting  needs  0  more examples
Cubism  needs  0  more examples
Early Renaissance  needs  289  more examples
Expressionism  needs  0  more examples
High Renaissance  needs  0  more examples
Impressionism  needs  0  more examples
Magic Realism  needs  0  more examples
Mannerism Late Renaissance   needs  0  more examples
Minimalism  needs  0  more examples
Naive Art Primitivism   needs  112  more examples
Neoclassicism  needs  0  more examples
Northern Renaissance  needs  0  more examples
Pop Art  needs  0  more examples
Post-Impressionism  needs  0  more examples
Realism  needs  0  more examples
Rococo  needs  0  more examples
Romanticism  needs  0  more examples
Surrealism  needs  0  more examples
Symbolism  needs  0  more examples
Ukiyo-e  needs  499  more examples


In [ ]:
#@title
def augment_and_save2(ART_DIR,AUG_PATH,style,parity,max_style_count,batch =50,img_batch = 200,save_percent = .7):
  new_path = os.path.join(AUG_PATH, style)
  style_path = os.path.join(ART_DIR, style)
  # Create new directory in augmented folder
  if not os.path.exists(new_path):
    os.makedirs(new_path)
    print('made new directory',new_path)

  # Change into current art style directory
  os.chdir(style_path)

  # List of all images for given style
  images = glob.glob("*.jpg") 
  print('leng images',len(images))
  if parity == 0: #Augmentations for the max class
    count = 1
    num_to_save = math.ceil(max_style_count*save_percent)
    print(num_to_save)
    for img in images:
      if count <= num_to_save:
        shutil.copy2(os.path.join(style_path,img),new_path)
      else:
        break
      count+=1
    
    # Subset of images to modify
    images = images[num_to_save-1:]

    # Number of images to augment
    remaining_images = max_style_count - count+1

    saved = 0
    while saved != remaining_images: 
      img_list = []
      img_names = []
      # Dont run past the right amount of augmented images
      if remaining_images-saved<img_batch:
        img_batch = remaining_images-saved
      for j in range(img_batch): 
        # Random list of images to be augmented 
        random_image = random.choice(images)
        im = cv2.imread(random_image)
        img_list.append(im)
        img_names.append(os.path.splitext(random_image)[0])

      if img_list:

        #Split into batches
        batch_list = [img_list[i:i + batch] for i in range(0, len(img_list), batch)]
        batch_names = [img_names[i:i + batch] for i in range(0, len(img_names), batch)]

        # Image augmentation
        for k in range(len(batch_list)):
          altered_images = seq(images = batch_list[k])

          # Store image in new aug folder
          for i in range(len(batch_list[k])):
            # filename = batch_names[k][i]

            # while os.path.exists(os.path.join(new_path,filename+'.jpg')):
            #   filename = filename +'aug'
            filename = str(saved)
            cv2.imwrite(os.path.join(new_path , filename+'.jpg'),altered_images[i])
            saved+=1

  else: # Save all copies of smaller classes, augment images until classes are balanced
    # for fname in images:
    #   # copying the files to the destination directory
    #   shutil.copy2(os.path.join(style_path,fname),new_path)

    print('\nAugmenting ',style,' images...')
    saved = 0
    while saved != parity: 
      img_list = []
      img_names = []
      # Dont run past the right amount of augmented images
      if parity-saved<img_batch:
        img_batch = parity-saved
      for j in range(img_batch): 
        # Random list of images to be augmented 
        random_image = random.choice(images)
        im = cv2.imread(random_image)
        img_list.append(im)
        img_names.append(os.path.splitext(random_image)[0])

      if img_list:

        #Split into batches
        batch_list = [img_list[i:i + batch] for i in range(0, len(img_list), batch)]
        batch_names = [img_names[i:i + batch] for i in range(0, len(img_names), batch)]

        # Image augmentation
        for k in range(len(batch_list)):
          altered_images = seq(images = batch_list[k])

          # Store image in new aug folder
          for i in range(len(batch_list[k])):
            filename = batch_names[k][i]
            filename = filename + 'aug'
            print(filename)
            while os.path.exists(os.path.join(new_path,filename+'.jpg')):
              filename = filename +'aug'
            
            cv2.imwrite(os.path.join(new_path , filename+'.jpg'),altered_images[i])
            saved+=1